# Подготовка датасет

Этот скрипт поможет подготовить Excel файл с данными о погрузке одного из типов грузов в период с 2018 по август (включительно 2024 года). Оригинальный файл был загружен с сайта Ростата.

Датасет предполагается использовать для решения задач по прогнозированию.

In [ ]:
#читаем файл с данными
import pandas as pd

# сохраним в этой переменной данные по всем грузам
data={}
start_year=2018
end_year=2025

#цикл с 2018 года по 2025 включительно 
for y in range(start_year,end_year+1):
    ys=str(y) # преобразуем год в строку
    # читаем файл листом номером года
    df=pd.read_excel("Pogruzka-08-2025.xlsx", sheet_name=ys, skiprows=3)
    df.rename(columns={'Unnamed: 0':'Груз'}, inplace=True)
    df.drop(index=[1], inplace=True)

    sheet={}
    for s in df.index:
        name=df.loc[s,'Груз']
        if pd.isna(name):
            break
        sheet[name]={}
        for c in df.columns[1:]:
            sheet[name][c]=df.loc[s,c]
    data[y]=sheet

print('Выберите тип груза из списка:')
list(sheet.keys())

In [ ]:
#скопируйте в качестве зачения переменной название груза в кавычках
gruz_name='химические и минеральные удобрения'

import calendar

def last_day_of_month(year, month):
    return calendar.monthrange(year, month)[1]

def convert_to_date(year, month):
    last_day = last_day_of_month(year, month)
    return pd.to_datetime(f"{year}-{month:02d}-{last_day:02d}")

# формируем датасет по одному грузу
one_item=[]

# собираем данные с 2018 года по 2025 включительно
for year in range(start_year,end_year+1):
    month=1 #номер стартового месяц и цикл по месяцам
    for one in data[year][gruz_name]:
        if 'год' in one: continue # не сохраняем стобец итого
        # собираем данные в список словарей
        one_item.append({'date':convert_to_date(year, month),  'value':data[year][gruz_name][one]})
        month+=1
df_one=pd.DataFrame(one_item)
df_one.dropna(inplace=True)
df_one.set_index('date', inplace=True)

#записываем результат в файл timeseries.xlsx
df_one.to_excel('timeseries.xlsx')